In [1]:
import requests
import pandas as pd
import datetime

In [2]:
import xmltodict
import json

In [3]:
import re

## Daten herunterladen von API und umwandeln

In [4]:
#Abfrage für alle Vorstösse einer Partei im gewählten Zeitrahmen (1.1.14 bis 1.1.24)
r = requests.get('http://www.gemeinderat-zuerich.ch/api/geschaeft/searchdetails?q=partei all "FDP" AND beginn_start > "2014-01-01 00:00:00" AND beginn_start < "2024-01-01 00:00:00" sortBy beginn_start/sort.ascending&l=de-CH')

In [5]:
#Die Arbeit mit den XML-Daten funktioniert schlecht, darum umwandeln in JSON
xml_data = r.text
data_dict = xmltodict.parse(xml_data)

## DataFrame mit Vorstössen erstellen

In [6]:
# Minidict mit den wichtigsten Angaben zu den Vorstössen. Da es nicht immer einen Mitunterzeichner gibt, braucht's einen Fail-Safe
vorstoesse = []

for hit in data_dict["SearchDetailResponse"]["Hit"]:
    titel = hit["Geschaeft"]["Titel"]
    geschaeftsart = hit["Geschaeft"]["Geschaeftsart"]
    ID = hit["Geschaeft"]["@OBJ_GUID"]
    datum = hit["Geschaeft"]["Beginn"]["Start"]["#text"]
    erstunterzeichner = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Name"]
    partei_erst = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Partei"]
    
    # Kontrollieren ob "Mitunterzeichner" existiert
    mitunterzeichner_data = hit["Geschaeft"].get("Mitunterzeichner")
    
    if mitunterzeichner_data:
        # Wenn es einen Mitunterzeichner gibt, dann Daten herunterladen
        mitunterzeichner = mitunterzeichner_data["KontaktGremium"]["Name"]
        partei_mit = mitunterzeichner_data["KontaktGremium"]["Partei"]
    else:
        # Wenn nicht, dann None einsetzen
        mitunterzeichner = None
        partei_mit = None
    
    status = hit["Geschaeft"]["Geschaeftsstatus"]
    
    mini_dict = {"Titel": titel,
                 "Geschäftsart": geschaeftsart,
                 "ID": ID,
                 "Datum": datum,
                 "Erstunterzeichner": erstunterzeichner,
                 "Partei Erstunterzeichner": partei_erst,
                 "Mitunterzeichner": mitunterzeichner,
                 "Partei Mitunterzeichner": partei_mit,
                 "Geschäftsstatus": status}
    
    vorstoesse.append(mini_dict)

In [7]:
df_partei = pd.DataFrame(vorstoesse)

In [8]:
df_partei

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus
0,"Erwerb des Koch-Areals von der UBS, Hintergrün...",Schriftliche Anfrage,958c7265ad524eb7ae344dc6c2416ca7,2014-01-08T00:00:00.000,Severin Pflüger,FDP,Albert Leiser,FDP,Abgeschlossen
1,"Gleichstellungsplan 2009–2013, Berücksichtigun...",Schriftliche Anfrage,3735062357634ac88f3699204b00e932,2014-01-15T00:00:00.000,Claudia Simon,FDP,Severin Pflüger,FDP,Abgeschlossen
2,"Restaurant Belvoir-Park, Verzicht auf den Bau ...",Postulat,96fe23887b7545299ff0562836335eef,2014-01-22T00:00:00.000,Michael Baumer,FDP,None,None,InBearbeitung
3,Einquartierung von Flüchtlingen an der Sonnenb...,Schriftliche Anfrage,32b22c754cf143a0804210f123845a6a,2014-01-22T00:00:00.000,Marc Bourgeois,FDP,Severin Pflüger,FDP,Abgeschlossen
4,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,e67c768f1c224330b68b363599ccb9be,2014-02-05T00:00:00.000,Marco Denoth,SP,Heinz F. Steger,FDP,Abgeschlossen
...,...,...,...,...,...,...,...,...,...
512,"Städtische Liegenschaftskäufe, Einordnung des ...",Schriftliche Anfrage,4b8e0c5364d5482cb6004550a10959c5,2023-11-15T00:00:00.000,Përparim Avdili,FDP,Michael Schmid,FDP,InBearbeitung
513,"Schäden durch die Aktion «Brings uf d'Strass»,...",Schriftliche Anfrage,f8e9c7cd80724351bdd4e9d5e81edfb7,2023-11-15T00:00:00.000,Jehuda Spielman,FDP,Flurin Capaul,FDP,InBearbeitung
514,Absetzung des Stücks «Bullet Zen» am Theater N...,Schriftliche Anfrage,9101ff7ddf9544e0ab91ceedd2f4850f,2023-11-22T00:00:00.000,Flurin Capaul,FDP,Yasmine Bourgeois,FDP,InBearbeitung
515,Mögliche Störaktionen durch aktive Personen im...,Schriftliche Anfrage,c35b41ebcab0415ab74a70f7121be70b,2023-11-29T00:00:00.000,Roger Meier,FDP,Claudio Zihlmann,FDP,InBearbeitung


## Resultate für Motionen und Postulate herunterladen

In [9]:
#Hiermit hole ich mir die Resultate für die Motionen und Postulate. 
#Es braucht einen doppelten Loop und Problembehebungsmassnahmen

entscheide = []

for hit in data_dict['SearchDetailResponse']['Hit']:
    # Kontrollieren, ob Ablaufschritte existiert und eine Liste ist
    ablaufschritte = hit['Geschaeft'].get('Ablaufschritte', {}).get('Aufgabe', [])
    
    if not isinstance(ablaufschritte, list):
        # Falls Aufgabe keine Liste ist, wird sie hier in eine Liste umgewandelt
        ablaufschritte = [ablaufschritte]
    
    for aufgabe in ablaufschritte:
        # Kontrollieren, ob AblaufschrittName existiert
        entscheid = aufgabe.get('AblaufschrittName')
        if entscheid is not None:
            entscheide.append({
                "HitID": hit["Geschaeft"]["@OBJ_GUID"],
                "Entscheid": entscheid,
            })

df_entscheid = pd.DataFrame(entscheide)

In [10]:
df_entscheid

,HitID,Entscheid
0,958c7265ad524eb7ae344dc6c2416ca7,"Eingang, Frist 3 Monate"
1,958c7265ad524eb7ae344dc6c2416ca7,"Stadtrat, Antwort"
2,958c7265ad524eb7ae344dc6c2416ca7,Kenntnisnahme
3,3735062357634ac88f3699204b00e932,"Stadtrat, Antwort"
4,3735062357634ac88f3699204b00e932,Kenntnisnahme
...,...,...
1806,f8e9c7cd80724351bdd4e9d5e81edfb7,"Eingang, Frist 3 Monate"
1807,f8e9c7cd80724351bdd4e9d5e81edfb7,"Stadtrat, Antwort"
1808,9101ff7ddf9544e0ab91ceedd2f4850f,"Eingang, Frist 3 Monate"
1809,c35b41ebcab0415ab74a70f7121be70b,"Eingang, Frist 3 Monate"


## Daten putzen

In [11]:
#Zu jedem Geschäft gibt es mehrere Verfahrensschritte. 
#Mich interessieren nur die Schlussentscheide, die ich mit den keywords heraushole

keywords = ["Überweisung", "Ablehnung", "Rückzug", "Umwandlung"]

# Ich kann ein Pattern kreieren mit meinen keyword, zusammengefügt werden sie mit dem OR operator (|)
pattern = '|'.join(map(re.escape, keywords))

# Jetzt kann ich das Pattern auf meine Entscheidspalte anwenden mit str.contains
df_entscheid_neu = df_entscheid[df_entscheid.Entscheid.str.contains(pattern, case=False, regex=True)]

In [12]:
df_entscheid_neu

,HitID,Entscheid
8,96fe23887b7545299ff0562836335eef,"Überweisung, Frist 24 Monate"
26,4940b23f724d4d6cb4a21aed8f043f63,"Überweisung, Frist 24 Monate"
34,b1aa3a1145e74412864318eadf3417b8,Rückzug
36,b1aa3a1145e74412864318eadf3417b8,"Stadtrat, Ablehnung"
46,e4d6bfe5ac3f477d874fb8c65662aabe,"Stadtrat, Ablehnung"
...,...,...
1776,95459311c22f42a98a183af40a643142,"Ablehnung, beantragt"
1779,e6995196dcd143e489937c9b153f4f92,"Ablehnung, beantragt"
1786,0ee00994db24493e85f6692a345a5263,"Ablehnung, beantragt"
1789,21355d5ae78e4f5cbe4bb875d8190ce0,"Ablehnung, beantragt"


In [13]:
# Das Problem ist noch nicht ganz gelöst, zwei Verfahrensschritte sind geblieben, die ich nicht brauche: 
exclude_values = ["Stadtrat, Ablehnung", "Ablehnung, beantragt"]

# Für das bereinigte DataFrame behalte ich nun alle Zeilen, welche die exclude-values nicht enthalten
df_entscheid_clean = df_entscheid_neu[~df_entscheid_neu['Entscheid'].isin(exclude_values)]

In [14]:
df_entscheid_clean

,HitID,Entscheid
8,96fe23887b7545299ff0562836335eef,"Überweisung, Frist 24 Monate"
26,4940b23f724d4d6cb4a21aed8f043f63,"Überweisung, Frist 24 Monate"
34,b1aa3a1145e74412864318eadf3417b8,Rückzug
47,e4d6bfe5ac3f477d874fb8c65662aabe,Ablehnung
55,731bc281e27f40829a886ac5a30261da,"Überweisung, Frist 24 Monate"
...,...,...
1714,d5f9308d7e8c4c11bb4aeedce24e1482,"Überweisung, Frist 24 Monate"
1725,550dcd568b4f466ba5a93bb2f1abfeb6,Ablehnung
1731,f542393e32934d738b15cdacc71d95c5,"Überweisung, Frist 24 Monate"
1747,5983ae001fad4dd18f0ddcbbb594ab6c,Ablehnung


In [15]:
#Nun sind noch verschiedene Varianten für Überweisung verblieben, die ich noch vereinheitliche und in neuer Spalte speichere
def clean(value):
    if 'Überweisung' in value:
        return 'überweisung'
    
    return value

#df_entscheid_clean["_entscheidbereinigt"] = df_entscheid_clean["Entscheid"].apply(clean)
df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']
df_entscheid_clean.loc[df_entscheid_clean['_entscheidbereinigt'].str.contains("Überweisung"), '_entscheidbereinigt'] = "Überweisung"
df_entscheid_clean

C:\Users\hudecj\AppData\Local\Temp\ipykernel_25128\3655237080.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']


,HitID,Entscheid,_entscheidbereinigt
8,96fe23887b7545299ff0562836335eef,"Überweisung, Frist 24 Monate",Überweisung
26,4940b23f724d4d6cb4a21aed8f043f63,"Überweisung, Frist 24 Monate",Überweisung
34,b1aa3a1145e74412864318eadf3417b8,Rückzug,Rückzug
47,e4d6bfe5ac3f477d874fb8c65662aabe,Ablehnung,Ablehnung
55,731bc281e27f40829a886ac5a30261da,"Überweisung, Frist 24 Monate",Überweisung
...,...,...,...
1714,d5f9308d7e8c4c11bb4aeedce24e1482,"Überweisung, Frist 24 Monate",Überweisung
1725,550dcd568b4f466ba5a93bb2f1abfeb6,Ablehnung,Ablehnung
1731,f542393e32934d738b15cdacc71d95c5,"Überweisung, Frist 24 Monate",Überweisung
1747,5983ae001fad4dd18f0ddcbbb594ab6c,Ablehnung,Ablehnung


In [16]:
#Hier führe ich die beiden Tabellen zusammen anhand der ID resp. HitID

df_neu = df_partei.copy()

df_partei_final = df_neu.merge(df_entscheid_clean, how='left', left_on='ID', right_on='HitID')[["Titel", "Geschäftsart", "ID", "Datum", "Erstunterzeichner", "Partei Erstunterzeichner", "Mitunterzeichner", "Partei Mitunterzeichner", "Geschäftsstatus", "Entscheid", '_entscheidbereinigt']]
df_partei_final

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,"Erwerb des Koch-Areals von der UBS, Hintergrün...",Schriftliche Anfrage,958c7265ad524eb7ae344dc6c2416ca7,2014-01-08T00:00:00.000,Severin Pflüger,FDP,Albert Leiser,FDP,Abgeschlossen,NaN,NaN
1,"Gleichstellungsplan 2009–2013, Berücksichtigun...",Schriftliche Anfrage,3735062357634ac88f3699204b00e932,2014-01-15T00:00:00.000,Claudia Simon,FDP,Severin Pflüger,FDP,Abgeschlossen,NaN,NaN
2,"Restaurant Belvoir-Park, Verzicht auf den Bau ...",Postulat,96fe23887b7545299ff0562836335eef,2014-01-22T00:00:00.000,Michael Baumer,FDP,None,None,InBearbeitung,"Überweisung, Frist 24 Monate",Überweisung
3,Einquartierung von Flüchtlingen an der Sonnenb...,Schriftliche Anfrage,32b22c754cf143a0804210f123845a6a,2014-01-22T00:00:00.000,Marc Bourgeois,FDP,Severin Pflüger,FDP,Abgeschlossen,NaN,NaN
4,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,e67c768f1c224330b68b363599ccb9be,2014-02-05T00:00:00.000,Marco Denoth,SP,Heinz F. Steger,FDP,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
512,"Städtische Liegenschaftskäufe, Einordnung des ...",Schriftliche Anfrage,4b8e0c5364d5482cb6004550a10959c5,2023-11-15T00:00:00.000,Përparim Avdili,FDP,Michael Schmid,FDP,InBearbeitung,NaN,NaN
513,"Schäden durch die Aktion «Brings uf d'Strass»,...",Schriftliche Anfrage,f8e9c7cd80724351bdd4e9d5e81edfb7,2023-11-15T00:00:00.000,Jehuda Spielman,FDP,Flurin Capaul,FDP,InBearbeitung,NaN,NaN
514,Absetzung des Stücks «Bullet Zen» am Theater N...,Schriftliche Anfrage,9101ff7ddf9544e0ab91ceedd2f4850f,2023-11-22T00:00:00.000,Flurin Capaul,FDP,Yasmine Bourgeois,FDP,InBearbeitung,NaN,NaN
515,Mögliche Störaktionen durch aktive Personen im...,Schriftliche Anfrage,c35b41ebcab0415ab74a70f7121be70b,2023-11-29T00:00:00.000,Roger Meier,FDP,Claudio Zihlmann,FDP,InBearbeitung,NaN,NaN


In [17]:
#Diese Resultate speichern
df_partei_final.to_csv('FDP_Vorstoesse_14_23_mit_Resultat.csv', index=False)

## Das Problem mit den umgewandelten Postulaten lösen

In [18]:
#Ein Problem bleibt noch: Bei Motionen, die in Postulate umgewandelt wurden, ist das Ergebnis nicht verzeichnet. 
#Weil es wenige Fälle sind, schaue ich das von Hand in den dazugehörigen PDF nach.
df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
31,Vermietung eines Anteils der Wohnungen an Sozi...,Motion,795afcc25c8e4e38b6c506c21a09b8cb,2015-06-24T00:00:00.000,Ursula Uttinger,FDP,Severin Pflüger,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
74,Aufhebung der Kap-Haltestelle Rebbergsteig der...,Motion,0bdaa4d9d6e44f578f746a9fe92c79c3,2016-08-24T00:00:00.000,Claudia Simon,FDP,Andreas Egli,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
105,Erarbeitung einer Vorlage für eine zukunftsfäh...,Motion,e2afad362e264c05940863f969f2895b,2017-06-21T00:00:00.000,Isabel Garcia,FDP,Michael Schmid,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
113,Erarbeitung der Grundlagen für eine Anbindung ...,Motion,5308dbd382e64c6d800268ce0a5dfd79,2017-07-12T00:00:00.000,Michael Baumer,FDP,Christoph Luchsinger,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
133,Erstellen einer Strategie für die Alters- und ...,Motion,20e582d6e370445daa13ef1e99cefd95,2018-01-17T00:00:00.000,Elisabeth Schoch,FDP,Albert Leiser,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
135,Einführung einer einheitlichen digitalen ID fü...,Motion,ade2db27c44e40c0927562f5aaa2292a,2018-01-31T00:00:00.000,Isabel Garcia,FDP,Corina Gredig,GLP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
147,Vergabe der gemeinnützigen Wohnungen an über 7...,Motion,c6ba76dc65f145d58cc274ef758dd218,2018-06-06T00:00:00.000,Elisabeth Schoch,FDP,Albert Leiser,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
202,Schrittweise Umsetzung einer «Smart School»-St...,Motion,4ae0f74066a24db79ff53fac784bfaf5,2019-03-13T00:00:00.000,Yasmine Bourgeois,FDP,Michael Schmid,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
226,Vollständige Deckung des Gasbedarfs der Stadt ...,Motion,559cba49e3c24a81a18eeaa87bd7fa77,2019-09-11T00:00:00.000,Guido Hüni,GLP,Sebastian Vogel,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
291,Realisierung eines Campus-Projekts auf dem Are...,Motion,d024c969e5594f35a8eb825cb0f85722,2020-09-23T00:00:00.000,Elisabeth Schoch,FDP,Corina Ursprung,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat


In [19]:
#Um schneller zum Ziel zu kommen, lasse ich mir die urls zu den Geschäften automatisch erstellen
df_umwandlung = df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

In [20]:
url_liste = []
base = "https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid="

for elem in df_umwandlung["ID"]:
    url_liste.append(base + elem)

In [21]:
url_liste

['https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=795afcc25c8e4e38b6c506c21a09b8cb',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=0bdaa4d9d6e44f578f746a9fe92c79c3',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=e2afad362e264c05940863f969f2895b',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=5308dbd382e64c6d800268ce0a5dfd79',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=20e582d6e370445daa13ef1e99cefd95',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=ade2db27c44e40c0927562f5aaa2292a',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=c6ba76dc65f145d58cc274ef758dd218',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=4ae0f74066a24db79ff53fac784bfaf5',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=559cba49e3c24a81a18eeaa87bd7fa77',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=d024c969e5594f35a8eb825cb0f85722',
 'https://www.gemein

In [22]:
#In einem ersten Schritt passe ich nun die umgewandelten Postulate an, die abgelehnt wurden
#Mit dem loc-Befehl kann man über die Index-Nr. die richtige Zeile anwählen und dann die Spalte
df_test = df_partei_final.copy()
df_test.loc[74, "_entscheidbereinigt"] = "Ablehnung"
df_test.loc[105, "_entscheidbereinigt"] = "Ablehnung"
df_test.loc[147, "_entscheidbereinigt"] = "Ablehnung"

In [23]:
#Ein Kontrollblick
pd.set_option('display.max_rows', None)
df_test

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,"Erwerb des Koch-Areals von der UBS, Hintergrün...",Schriftliche Anfrage,958c7265ad524eb7ae344dc6c2416ca7,2014-01-08T00:00:00.000,Severin Pflüger,FDP,Albert Leiser,FDP,Abgeschlossen,NaN,NaN
1,"Gleichstellungsplan 2009–2013, Berücksichtigun...",Schriftliche Anfrage,3735062357634ac88f3699204b00e932,2014-01-15T00:00:00.000,Claudia Simon,FDP,Severin Pflüger,FDP,Abgeschlossen,NaN,NaN
2,"Restaurant Belvoir-Park, Verzicht auf den Bau ...",Postulat,96fe23887b7545299ff0562836335eef,2014-01-22T00:00:00.000,Michael Baumer,FDP,None,None,InBearbeitung,"Überweisung, Frist 24 Monate",Überweisung
3,Einquartierung von Flüchtlingen an der Sonnenb...,Schriftliche Anfrage,32b22c754cf143a0804210f123845a6a,2014-01-22T00:00:00.000,Marc Bourgeois,FDP,Severin Pflüger,FDP,Abgeschlossen,NaN,NaN
4,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,e67c768f1c224330b68b363599ccb9be,2014-02-05T00:00:00.000,Marco Denoth,SP,Heinz F. Steger,FDP,Abgeschlossen,NaN,NaN
5,"Neuausrichtung des Strauhofs, Abklärungen bezü...",Schriftliche Anfrage,76de5e7837a543188230f7256875e175,2014-03-12T00:00:00.000,Claudia Simon,FDP,Severin Pflüger,FDP,Abgeschlossen,NaN,NaN
6,Vergabepraxis günstiger Wohnungen durch die St...,Schriftliche Anfrage,f87e14210ae1446db6773537291748aa,2014-03-12T00:00:00.000,Albert Leiser,FDP,Markus Hungerbühler,CVP,Abgeschlossen,NaN,NaN
7,Einführung einer Jugendinitiative als Instrume...,Postulat,4940b23f724d4d6cb4a21aed8f043f63,2014-03-19T00:00:00.000,Isabel Garcia,FDP,Matthias Wiesmann,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
8,"Sechseläutenplatz, Nutzung für politische Vera...",Schriftliche Anfrage,ceb42fa107044b5d8d26143420d8daeb,2014-03-26T00:00:00.000,Urs Egger,FDP,None,None,Abgeschlossen,NaN,NaN
9,"Energieplanungsbericht des Kantons Zürich, Fol...",Schriftliche Anfrage,f3f9395c83d84f62ade89d5fb5a4895e,2014-04-16T00:00:00.000,Helen Glaser,SP,Alexander Jäger,FDP,Abgeschlossen,NaN,NaN


In [24]:
pd.reset_option('display.max_rows')

In [25]:
#Nun kann ich alle übrigen umgewandelten Postulate zu "Überweisung" ändern
df_test['_entscheidbereinigt'] = df_test['_entscheidbereinigt'].replace('Umwandlung in Postulat', 'Überweisung')

In [26]:
#Diese Resultate speichern
df_test.to_csv('FDP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv', index=False)